In [1]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   char processor_name[MPI_MAX_PROCESSOR_NAME];
   int name_len;
   MPI_Get_processor_name(processor_name, &name_len);

   std::cout << "Hello world from processor " << processor_name << ", rank " << myid << " out of " << numprocs << " processors" << std::endl;

   MPI_Finalize();
   return 0;
}

Writing main.cpp


In [2]:
!mpiCC main.cpp -o main.ex

Undefined symbols for architecture arm64:
  "std::__1::locale::use_facet(std::__1::locale::id&) const", referenced from:
      std::__1::ctype<char> const& std::__1::use_facet[abi:v15006]<std::__1::ctype<char>>(std::__1::locale const&) in main-1aba56.o
  "std::__1::ios_base::getloc() const", referenced from:
      std::__1::basic_ios<char, std::__1::char_traits<char>>::widen[abi:v15006](char) const in main-1aba56.o
  "std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>::__init(unsigned long, char)", referenced from:
      std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>::basic_string[abi:v15006](unsigned long, char) in main-1aba56.o
  "std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>::~basic_string()", referenced from:
      std::__1::ostreambuf_iterator<char, std::__1::char_traits<char>> std::__1::__pad_and_output<char, std::__1::char_traits<char>>(std::__1::ostreambuf_iterator<cha

In [4]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

--------------------------------------------------------------------------
prterun was unable to launch the specified application as it lacked
permissions to execute an executable:

   Executable: ./main.ex
   Node: Mitchells-MacBook-Pro

while attempting to start process rank 0.
--------------------------------------------------------------------------


In [5]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>
#include <omp.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(argc != 2)
   {
      omp_set_num_threads(2);
   }
   else
   {
      omp_set_num_threads(atoi(argv[1]));
   }
   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
      std::cout << "Number of threads: " << omp_get_max_threads() << std::endl;
   }

   int *x = new int[numprocs];
   for(int i = 0; i < numprocs; i++)
   {
      x[i] = i + myid * numprocs;
   }

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         #pragma omp parallel
         {
            int tid = omp_get_thread_num();
            #pragma omp critical
            {
               std::cout << "Processor " << myid << " thread " << tid << " has x = ";
               for(int i = 0; i < numprocs; i++)
               {
                  std::cout << x[i] << " ";
               }
               std::cout << std::endl;
            }
         }
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   sleep(1);

   for(int i = 0; i < numprocs; i++)
   {
      if(myid == i)
      {
         #pragma omp parallel
         {
            int tid = omp_get_thread_num();
            #pragma omp critical
            {
               std::cout << "x[0] on processor " << myid << " thread " << tid << " is " << x[0] << std::endl;
            }
         }
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

Overwriting main.cpp


In [ ]:
!mpiCC main.cpp -o main.ex -fopenmp

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   int x = myid;

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before Bcast, processor " << myid << " has x = " << x << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Bcast(&x, 1, MPI_INT, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Bcast, processor " << myid << " has x = " << x << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   int x = myid;
   int sum = 10086;

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before Reduce, processor " << myid << " has x = " << x << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Reduce(&x, &sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Reduce, processor " << myid << " has sum = " << sum << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }
   MPI_Barrier(MPI_COMM_WORLD);

   MPI_Finalize();

   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
   }

   int *x = new int[numprocs];
   for(int i = 0; i < numprocs; i++)
   {
      x[i] = i + myid * numprocs;
   }

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before all-to-all broadcast, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Alltoall(MPI_IN_PLACE, 1, MPI_INT, x, 1, MPI_INT, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After all-to-all broadcast, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   int x = myid;
   int sum = 10086;

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before Allreduce, processor " << myid << " has x = " << x << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Allreduce(&x, &sum, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Allreduce, processor " << myid << " has sum = " << sum << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }
   MPI_Barrier(MPI_COMM_WORLD);

   MPI_Finalize();

   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
   }

   int *x = new int[numprocs];
   for(int i = 0; i < numprocs; i++)
   {
      x[i] = i + myid * numprocs;
   }

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before Scatter, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Scatter(x, 1, MPI_INT, x, 1, MPI_INT, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Scatter, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex

In [ ]:
%%writefile main.cpp
#include <mpi.h>
#include <iostream>
#include <unistd.h>

int main(int argc, char** argv) {
   MPI_Init(&argc, &argv);

   int myid, numprocs;
   MPI_Comm_size(MPI_COMM_WORLD, &numprocs);
   MPI_Comm_rank(MPI_COMM_WORLD, &myid);

   if(myid == 0)
   {
      std::cout << "Number of processors: " << numprocs << std::endl;
   }

   int *x = new int[numprocs];
   for(int i = 0; i < numprocs; i++)
   {
      x[i] = i + myid * numprocs;
   }

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "Before Gather, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Gather(x, 1, MPI_INT, x, 1, MPI_INT, 0, MPI_COMM_WORLD);

   for(int id = 0 ; id < numprocs; id++)
   {
      if(myid == id)
      {
         std::cout << "After Gather, processor " << myid << " has x = ";
         for(int i = 0; i < numprocs; i++)
         {
            std::cout << x[i] << " ";
         }
         std::cout << std::endl;
      }
      MPI_Barrier(MPI_COMM_WORLD);
   }

   MPI_Finalize();
   return 0;
}

In [ ]:
!mpiCC main.cpp -o main.ex

In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 ./main.ex